In [26]:
import os
import json
from pathlib import Path
import shutil

In [27]:
def extract_categories_from_shapes(shapes):
    """
    Extrae las categorías únicas de las anotaciones en el archivo JSON.
    """
    categories = set()
    for shape in shapes:
        categories.add(shape["label"])
    return list(categories)

def create_coco_categories(categories):
    """
    Crea la lista de categorías en el formato COCO, asignando un ID a cada categoría.
    """
    return [{"id": idx + 1, "name": name} for idx, name in enumerate(categories)]

def create_coco_image_info(image_path, image_height, image_width, image_id):
    """
    Crea la entrada de información de imagen en formato COCO.
    """
    return {
        "file_name": image_path,
        "height": image_height,
        "width": image_width,
        "id": image_id
    }

def convert_shape_to_coco_annotation(shape, category_id, image_id, annotation_id):
    """
    Convierte una anotación de `Labelme` a la estructura de anotación en formato COCO.
    """
    points = shape["points"]
    flattened_points = [coord for point in points for coord in point]
    
    # Calcula el bounding box en el formato COCO [x_min, y_min, width, height]
    x_coords = [p[0] for p in points]
    y_coords = [p[1] for p in points]
    bbox = [
        min(x_coords),
        min(y_coords),
        max(x_coords) - min(x_coords),
        max(y_coords) - min(y_coords)
    ]

    return {
        "id": annotation_id,
        "image_id": image_id,
        "category_id": category_id,
        "segmentation": [flattened_points],
        "area": bbox[2] * bbox[3],  # Área aproximada del bounding box
        "bbox": bbox,
        "iscrowd": 0
    }

def labelme_to_coco(labelme_data, categories_dict, image_id):
    """
    Convierte un archivo JSON de `Labelme` a un diccionario en formato COCO.
    """
    coco_format = {
        "images": [],
        "annotations": [],
        "categories": create_coco_categories(categories_dict.keys())
    }

    # Agrega la información de la imagen
    image_info = create_coco_image_info(
        labelme_data["imagePath"],
        labelme_data["imageHeight"],
        labelme_data["imageWidth"],
        image_id
    )
    coco_format["images"].append(image_info)

    # Procesa cada anotación y la convierte a formato COCO
    annotation_id = 1
    if "shapes" in labelme_data:
        for shape in labelme_data["shapes"]:
            if shape["shape_type"] == "polygon":
                label = shape["label"]
                # Verificar si la etiqueta está en el diccionario de categorías
                if label in categories_dict:
                    category_id = categories_dict[label]
                    annotation = convert_shape_to_coco_annotation(shape, category_id, image_id, annotation_id)
                    coco_format["annotations"].append(annotation)
                    annotation_id += 1
                else:
                    print(f"Advertencia: la etiqueta '{label}' no está en categories_dict. Verifica las categorías.")
                    print("categories_dict:", categories_dict)
    return coco_format

def copy_image_to_output_folder(image_name, input_folder, output_folder):
    """
    Copia la imagen original al nuevo directorio de salida.
    """
    image_path = os.path.join(input_folder, image_name)
    if os.path.exists(image_path):
        shutil.copy(image_path, output_folder)
        print(f"Imagen {image_name} copiada al directorio de salida.")
    else:
        print(f"Advertencia: La imagen {image_name} no se encontró en el directorio de entrada.")

def process_all_json_files(input_folder, output_folder):
    """
    Procesa todos los archivos JSON en un directorio y los convierte a formato COCO.
    """
    os.makedirs(output_folder, exist_ok=True)
    json_files = list(Path(input_folder).rglob("*.json"))
    
    # Crear un diccionario global de categorías (etiquetas únicas con IDs)
    all_categories = set()
    for json_file in json_files:
        with open(json_file, 'r') as f:
            labelme_data = json.load(f)
        # Verificar si el archivo contiene la clave "shapes"
        if "shapes" in labelme_data:
            all_categories.update(extract_categories_from_shapes(labelme_data["shapes"]))
    categories_dict = {name: idx + 1 for idx, name in enumerate(all_categories)}

    print("Diccionario de categorías (categories_dict):", categories_dict)

    # Procesar cada archivo JSON y convertirlo a formato COCO
    image_id = 1
    for json_file in json_files:
        with open(json_file, 'r') as f:
            labelme_data = json.load(f)

        # Copiar la imagen original al directorio de salida
        copy_image_to_output_folder(labelme_data["imagePath"], input_folder, output_folder)

        # Convertir a formato COCO si contiene la clave "shapes"
        if "shapes" in labelme_data:
            coco_data = labelme_to_coco(labelme_data, categories_dict, image_id)

            # Guardar el archivo en formato COCO con el mismo nombre que el original
            output_path = os.path.join(output_folder, json_file.name)

            with open(output_path, 'w') as f:
                json.dump(coco_data, f, indent=4)
            print(f"Archivo {json_file.name} guardado en formato COCO en el directorio de salida.")

            # Incrementar el ID de imagen para el siguiente archivo
            image_id += 1

In [28]:
# Carpetas de entrada y salida
input_folder = "augmented"
output_folder = "output_yolov_coco"

# Ejecutar la función
process_all_json_files(input_folder, output_folder)

Diccionario de categorías (categories_dict): {'maduro': 1, 'Biche': 2, 'Maduro': 3, 'rojo': 4}
Imagen arandano_aug_1.jpg copiada al directorio de salida.
Archivo arandano_aug_1.json guardado en formato COCO en el directorio de salida.
Imagen arandano_aug_2.jpg copiada al directorio de salida.
Archivo arandano_aug_2.json guardado en formato COCO en el directorio de salida.
Imagen arandano_aug_29.jpg copiada al directorio de salida.
Archivo arandano_aug_29.json guardado en formato COCO en el directorio de salida.
Imagen arandano_aug_3.jpg copiada al directorio de salida.
Archivo arandano_aug_3.json guardado en formato COCO en el directorio de salida.
Imagen arandano_aug_4.jpg copiada al directorio de salida.
Archivo arandano_aug_4.json guardado en formato COCO en el directorio de salida.
Imagen arandano_aug_5.jpg copiada al directorio de salida.
Archivo arandano_aug_5.json guardado en formato COCO en el directorio de salida.
Imagen arandano_aug_6.jpg copiada al directorio de salida.
Arch